# Import

In [14]:
import pandas as pd
import numpy as np

import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
from shapely.geometry import Point
from shapely.geometry import LineString
import json
from pyproj import CRS

pd.set_option('display.max_columns', None)

In [ ]:
# Data pre-processing

In [23]:
df_num = pd.read_csv("./data_HWJ/천안시_노선번호목록.csv")
df_stops = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")
df_info = pd.read_csv("./data_HWJ/천안시_노선정보항목.csv", sep='\t')

In [28]:
df_num.head()

,종점,막차시간,노선ID,노선번호,노선유형,기점,첫차시간
0,종합터미널,1900.0,CAB285000267,980,일반버스,온양온천역 유엘시티,600.0
1,고속버스터미널,2010.0,CAB285000268,980,일반버스,고속터미널,650.0
2,종합터미널,1905.0,CAB285000269,981,일반버스,포스코아파트 정문,1315.0
3,포스코아파트 정문,1955.0,CAB285000270,981,일반버스,고속터미널,1405.0
4,월랑1리 마을회관,1100.0,CAB285000272,982,일반버스,종합터미널,655.0


In [29]:
df_stops.head()

,정류소 Y좌표,정류소 X좌표,정류소ID,정류소명,정류소번호,정류소순번,노선ID
0,36.78333,127.00490,CAB288000744,온양온천역 유엘시티,744,1,CAB285000267
1,36.78589,127.00661,CAB288000737,아고오거리,737,2,CAB285000267
2,36.78514,127.01044,CAB288000738,한올고등학교,738,3,CAB285000267
3,36.78428,127.01506,CAB288000715,온양터미널,808,4,CAB285000267
4,36.78777,127.01560,CAB288000697,아산소방서,697,5,CAB285000267


In [34]:
df_info.head()

,종점,막차시간,배차간격(평일),노선ID,노선번호,노선유형,기점,첫차시간
0,종합터미널,1900.0,195.0,CAB285000267,980,일반버스,온양온천역 유엘시티,600.0
1,고속버스터미널,2010.0,200.0,CAB285000268,980,일반버스,고속터미널,650.0
2,종합터미널,1905.0,0.0,CAB285000269,981,일반버스,포스코아파트 정문,1315.0
3,포스코아파트 정문,1955.0,0.0,CAB285000270,981,일반버스,고속터미널,1405.0
4,월랑1리 마을회관,1100.0,245.0,CAB285000272,982,일반버스,종합터미널,655.0


In [ ]:
df_stops.info()

: 

### plotly - 노선별 경유 정류소 목록 표시

In [21]:
df = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")

df.columns

fig = px.scatter_map(
    df,
    lat="정류소 Y좌표",
    lon="정류소 X좌표",
    color="노선ID",
    hover_name="정류소명", # 마우스 오버 시 표시한 텍스트
    # hover_data={},
    # text="text",
    zoom=11,
    height=650,
    );
 # carto-positron : 무료, 지도 배경 스타일 지정
fig.update_layout(map_style="carto-positron", margin={"r":0,"t":0,"l":0,"b":0})
fig.show();

In [19]:
df = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")

df.columns

fig = px.scatter_map(
    df,
    lat="정류소 Y좌표",
    lon="정류소 X좌표",
    color="노선ID",
    hover_name="정류소명", # 마우스 오버 시 표시한 텍스트
    # hover_data={},
    # text="text",
    zoom=11,
    height=650,
    );

fig.update_layout(map_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
fig.show();

### shp 파일로 Choropleth Mapbox 생성

In [ ]:
### shp 파일
# .shp 불러오기 → 좌표계 변환 → .geojson 저장
# CSV 불러오기 → 자치구별 데이터 집계
# GeoJSON과 CSV의 키값 맞추기
# Plotly로 색깔 입힌 지도(Choropleth Mapbox) 생성

# 1. 읍면동 경계 shp 파일 불러오기, 좌표계 변환
gdf = gpd.read_file("./data_HWJ/map_data/emd.shp", encoding="cp949")
print("원본 좌표계:", gdf.crs)

gdf.crs = "EPSG:5179"  # 원본 좌표계 지정 (예: UTM-K)
gdf = gdf.to_crs(epsg=4326)  # WGS84 위경도로 변환

# 2. 정류소 CSV 불러오기
df = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")

# 3. 정류소 DataFrame에 위도,경도 기준 Point geometry 생성 (경도=X, 위도=Y)
geometry = [Point(xy) for xy in zip(df['정류소 X좌표'], df['정류소 Y좌표'])]
gdf_stops = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# 4. 공간 조인 (정류소 포인트가 포함된 읍면동 폴리곤 가져오기)
gdf_joined = gpd.sjoin(gdf_stops, gdf, how='left', predicate='within')

# 5. 읍면동명 컬럼명 확인 (예: 'EMD_KOR_NM')와 필요한 컬럼 추출
print(gdf_joined.columns)
# 일반적으로 읍면동 명 컬럼이 'EMD_KOR_NM'일 가능성 높음

# 6. 읍면동별 정류소 수 집계
agg_df = gdf_joined.groupby('EMD_KOR_NM').size().reset_index(name='정류소수')
print(agg_df.head())

In [ ]:
# 7. GeoJSON 파일로 저장 (Plotly용)
gdf.to_file("./data_HWJ/map_data/mapfile.geojson", driver="GeoJSON")

with open('./data_HWJ/map_data/mapfile.geojson', encoding='utf-8') as f:
    geojson_data = json.load(f)

# 8. Plotly Choropleth Mapbox 시각화
fig = px.choropleth_map(
    agg_df,
    geojson=geojson_data,
    locations='EMD_KOR_NM',
    featureidkey='properties.EMD_KOR_NM',
    color='정류소수',
    color_continuous_scale='Viridis',
    map_style='carto-positron',
    center={'lat': 36.8, 'lon': 127.1},  # 천안시 중심 좌표 설정
    zoom=10,
    opacity=0.6,
    labels={'정류소수': '정류소 개수'},
    title='읍면동별 정류소 개수'
)

# open-street-map
# carto-positron

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

In [ ]:
# 7. GeoJSON 파일로 저장 (Plotly용)
gdf.to_file("./data_HWJ/map_data/mapfile.geojson", driver="GeoJSON")

with open('./data_HWJ/map_data/mapfile.geojson', encoding='utf-8') as f:
    geojson_data = json.load(f)

# 8. Plotly Choropleth Mapbox 시각화
fig = px.choropleth_map(
    agg_df,
    geojson=geojson_data,
    locations='EMD_KOR_NM',
    featureidkey='properties.EMD_KOR_NM',
    color='정류소수',
    color_continuous_scale='Viridis',
    map_style='open-street-map',
    center={'lat': 36.8, 'lon': 127.1},  # 천안시 중심 좌표 설정
    zoom=10,
    opacity=0.6,
    labels={'정류소수': '정류소 개수'},
    title='읍면동별 정류소 개수'
)

# open-street-map
# carto-positron

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

### 노선별 line string 시각화

In [ ]:
# 1. 정류소 데이터 불러오기
df = pd.read_csv('./data_HWJ/천안시_노선별경유정류소목록.csv')

# 2. 노선별 정류소를 순서대로 정렬
df_sorted = df.sort_values(by=['노선ID', '정류소순번'])

# 3. 노선별 LineString 생성 함수
def create_linestring(group):
    points = list(zip(group['정류소 X좌표'], group['정류소 Y좌표']))
    return LineString(points)

# 4. 노선별 LineString GeoDataFrame 생성
lines = df_sorted.groupby('노선ID').apply(create_linestring).reset_index()
lines.columns = ['노선ID', 'geometry']
gdf_lines = gpd.GeoDataFrame(lines, geometry='geometry', crs='EPSG:4326')

# 5. Plotly로 시각화
fig = go.Figure()

colors = px.colors.qualitative.Dark24  # 색상 팔레트

for i, (route_id, row) in enumerate(gdf_lines.iterrows()):
    x, y = row.geometry.xy
    lon = list(x)  # array -> list 변환
    lat = list(y)
    fig.add_trace(go.Scattermapbox(
        lon=lon,
        lat=lat,
        mode='lines+markers',
        name=route_id,
        line=dict(width=1, color=colors[i % len(colors)]),
        marker=dict(size=5)
    ))

fig.update_layout(
    mapbox_style='carto-positron',
    mapbox_center={"lat": 36.8, "lon": 127.1},  # 천안시 중심 좌표
    mapbox_zoom=10,
    margin={"r":0,"t":0,"l":0,"b":0},
    legend_title_text='노선ID별',
    title='노선별 정류소 시각화'
    
)

fig.show()


In [ ]:
1. 도보거리
2. 저상버스 적용유무
3. 지나가는 노선 
4. 인구밀도
5. 배차시간

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px

# --------------------
# 1. 데이터 불러오기
# --------------------
stops_df = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")
facilities_df = pd.read_csv("./data_HWJ/장애인_재활시설.csv")
gdf_region = gpd.read_file("./data_HWJ/map_data/mapfile.geojson")

# --------------------
# 2. GeoDataFrame 변환 & 좌표계 통일
# --------------------
# 정류소
gdf_stops = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df["정류소 X좌표"], stops_df["정류소 Y좌표"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# 복지시설
gdf_facilities = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df["경도"], facilities_df["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# 읍면동 경계
if gdf_region.crs is None:
    gdf_region = gdf_region.set_crs(epsg=4326)
gdf_region = gdf_region.to_crs(epsg=5179)

# --------------------
# 3. 정류소 500m Buffer & 시설 교차 여부
# --------------------
gdf_stops["geometry_buffer"] = gdf_stops.buffer(500)
gdf_stops["복지시설500m이내"] = gdf_stops["geometry_buffer"].apply(
    lambda buf: gdf_facilities.intersects(buf).any()
).astype(int)

# --------------------
# 4. 읍면동 Spatial Join
# --------------------
if "index_right" in gdf_stops.columns:
    gdf_stops = gdf_stops.drop(columns="index_right")

gdf_stops = gpd.sjoin(
    gdf_stops,
    gdf_region[["EMD_KOR_NM", "geometry"]],
    how="left",
    predicate="intersects"
)

gdf_stops = gdf_stops.loc[:, ~gdf_stops.columns.duplicated()]

# --------------------
# 5. 읍면동별 집계
# --------------------
result = (
    gdf_stops.groupby("EMD_KOR_NM")
    .agg(
        총정류소수=("정류소명", "count"),
        복지시설500m이내=("복지시설500m이내", "sum")
    )
    .reset_index()
)
result["비율(%)"] = (result["복지시설500m이내"] / result["총정류소수"] * 100).round(1)
result["비율(%)"] = result["비율(%)"].fillna(0)

# --------------------
# 6. Choropleth 시각화 (Mapbox 없이)
# --------------------
# 지도용 좌표계: WGS84(EPSG:4326)
gdf_region_plot = gdf_region.to_crs(epsg=4326)
gdf_region_plot = gdf_region_plot.merge(result, on="EMD_KOR_NM", how="left")

fig = px.choropleth(
    gdf_region_plot,
    geojson=gdf_region_plot.__geo_interface__,
    locations="EMD_KOR_NM",
    featureidkey="properties.EMD_KOR_NM",
    color="비율(%)",
    hover_data=["EMD_KOR_NM", "총정류소수", "복지시설500m이내", "비율(%)"],
    color_continuous_scale="Blues",
    labels={"비율(%)": "500m 이내 복지시설 비율"}
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title="천안시 읍면동별 정류소 500m 이내 복지시설 접근성",
    margin={"r":0,"t":30,"l":0,"b":0}
)

fig.show()

In [ ]:
# --------------------
# 1. 데이터 불러오기
# --------------------
stops_df = pd.read_csv("./data_HWJ/천안시_노선별경유정류소목록.csv")
facilities_df = pd.read_csv("./data_HWJ/장애인_재활시설.csv")
gdf_region = gpd.read_file("./data_HWJ/map_data/mapfile.geojson")

# --------------------
# 2. GeoDataFrame 변환 & 좌표계 통일
# --------------------
# 정류소
gdf_stops = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df["정류소 X좌표"], stops_df["정류소 Y좌표"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# 복지시설
gdf_facilities = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df["경도"], facilities_df["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# 읍면동 경계
if gdf_region.crs is None:
    gdf_region = gdf_region.set_crs(epsg=4326)
gdf_region = gdf_region.to_crs(epsg=5179)

# --------------------
# 3. 정류소 500m Buffer & 시설 교차 여부
# --------------------
gdf_stops["geometry_buffer"] = gdf_stops.buffer(500)
gdf_stops["복지시설500m이내"] = gdf_stops["geometry_buffer"].apply(
    lambda buf: gdf_facilities.intersects(buf).any()
).astype(int)

# --------------------
# 4. 읍면동 Spatial Join
# --------------------
if "index_right" in gdf_stops.columns:
    gdf_stops = gdf_stops.drop(columns="index_right")

gdf_stops = gpd.sjoin(
    gdf_stops,
    gdf_region[["EMD_KOR_NM", "geometry"]],
    how="left",
    predicate="intersects"
)

gdf_stops = gdf_stops.loc[:, ~gdf_stops.columns.duplicated()]

# --------------------
# 5. 읍면동별 집계
# --------------------
result = (
    gdf_stops.groupby("EMD_KOR_NM")
    .agg(
        총정류소수=("정류소명", "count"),
        복지시설500m이내=("복지시설500m이내", "sum")
    )
    .reset_index()
)
result["비율(%)"] = (result["복지시설500m이내"] / result["총정류소수"] * 100).round(1)
result["비율(%)"] = result["비율(%)"].fillna(0)

# --------------------
# 6. Choropleth 시각화 (Mapbox 없이)
# --------------------
# 지도용 좌표계: WGS84(EPSG:4326)
gdf_region_plot = gdf_region.to_crs(epsg=4326)
gdf_region_plot = gdf_region_plot.merge(result, on="EMD_KOR_NM", how="left")

fig = px.choropleth_map(
    gdf_region_plot,
    geojson=gdf_region_plot.__geo_interface__,
    locations="EMD_KOR_NM",
    featureidkey="properties.EMD_KOR_NM",
    color="비율(%)",
    hover_data=["EMD_KOR_NM", "총정류소수", "복지시설500m이내", "비율(%)"],
    color_continuous_scale="Blues",
    labels={"비율(%)": "500m 이내 복지시설 비율"}
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title="천안시 읍면동별 정류소 500m 이내 복지시설 접근성",
    margin={"r":0,"t":30,"l":0,"b":0}
)

fig.show()

In [ ]:
gdf_region